# **Redução de Dimensionalidade - Módulo 5, Notebook 2/3**

---

## Índice

1. [Introdução](#introducao)
2. [A Maldição da Dimensionalidade](#maldicao)
3. [Projeção vs. Manifold Learning](#projecao-manifold)
4. [PCA - Principal Component Analysis](#pca)
   - [Definição e Intuição](#pca-intuicao)
   - [Formulação Matemática](#pca-matematica)
   - [Implementação com Scikit-Learn](#pca-implementacao)
5. [LLE - Locally Linear Embedding](#lle)
   - [Definição e Intuição](#lle-intuicao)
   - [Implementação e Comparação com PCA](#lle-implementacao)
6. [Outros Métodos de Redução de Dimensionalidade](#outros-metodos)

---

<a id='introducao'></a>
## 1. Introdução

Quando trabalhamos com dados do mundo real, frequentemente nos deparamos com datasets que possuem dezenas, centenas ou até milhares de features. Embora mais informação pareça sempre melhor, há um preço a pagar: a **maldição da dimensionalidade**.

Neste tópico, vamos explorar:
- Por que muitas features podem prejudicar nossos modelos
- Como reduzir dimensionalidade preservando informação relevante
- Dois métodos: PCA (projeção) e LLE (manifold learning)
- Quando usar cada abordagem

In [ ]:
# ======= IMPORTS =======
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_digits, make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Configuração visual
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

<a id='maldicao'></a>
## 2. A Maldição da Dimensionalidade

Quando o número de features cresce, o espaço de dados se torna esparso e a noção de vizinhança se dissolve: as distâncias ficam parecidas entre si e perdem poder explicativo. Isso acontece porque cada nova dimensão adiciona um critério extra para que dois pontos sejam considerados próximos. Se as variáveis extras carregam pouco ou nenhum sinal, elas diluem a influência das variáveis úteis e fazem o ruído dominar a métrica de distância, prejudicando modelos baseados em proximidade.

Pense no problema de prever renda. Com poucas variáveis relevantes, como escolaridade, é fácil encontrar indivíduos comparáveis. Ao acrescentar atributos pouco relacionados, como número de cachorros ou estilo musical, a distância entre essas mesmas pessoas aumenta - não porque o fenômeno mudou, mas porque exigimos semelhança em critérios que não trazem informação. Com dezenas ou centenas de variáveis irrelevantes, todos os pontos passam a parecer igualmente distantes.

Uma resposta intuitiva seria sugerir simplesmente “coletar mais dados”. O problema é que, em alta dimensão, a quantidade de observações necessária para manter uma densidade mínima cresce exponencialmente com o número de features. Para preservar a mesma resolução do espaço - isto é, para continuar encontrando pontos minimamente próximos - o volume a ser preenchido explode. Em dimensões elevadas, o número de amostras exigido rapidamente ultrapassa qualquer possibilidade prática, tornando-se astronômico. Por isso, a solução não é apenas mais dados, é necessário reduzir a dimensionalidade do problema, seja por meio da seleção de variáveis realmente informativas, seja pela construção de representações mais compactas que preservem o sinal relevante.

<a id='projecao-manifold'></a>
## 3. Projeção vs. Manifold Learning

Existem duas abordagens principais para reduzir dimensionalidade: **projeção** e **manifold learning**. A primeira busca subespaços lineares que capturam máxima variância; a segunda tenta desenrolar estruturas não-lineares preservando relações locais.

### 3.1 Projeção

A ideia é encontrar direções no espaço original ao longo das quais os dados variam mais e descartar as direções com pouca variação. Imagine uma nuvem de pontos 3D que na prática forma um disco achatado: podemos projetá-la em um plano 2D sem perder muito, pois a terceira dimensão carrega pouca informação.

O método mais conhecido concerteza é o **PCA (Principal Component Analysis)**. Ele é simples, rápido e funciona bem quando os dados variam principalmente em direções lineares. A transformação é explícita e reversível, permitindo aplicar a mesma projeção a novos dados. A limitação é que assume relações lineares e falha quando a estrutura subjacente é complexa ou não-linear.

Vamos visualizar essa ideia com um exemplo simples.

In [ ]:
# ======= EXEMPLO: PROJEÇÃO - MAIOR VS MENOR VARIÂNCIA =======

# Gerar dados 3D com tendência clara e variâncias muito diferentes
np.random.seed(42)
n = 400

# Criar dados com forte tendência linear em uma direção
t = np.linspace(0, 4*np.pi, n)
X_proj = np.zeros((n, 3))
X_proj[:, 0] = 3 * t + np.random.randn(n) * 0.8      # Forte variância + tendência no eixo X
X_proj[:, 1] = 1.2 * np.sin(t) + np.random.randn(n) * 0.5  # Variância média no eixo Y
X_proj[:, 2] = 0.15 * np.cos(t*2) + np.random.randn(n) * 0.15  # Pouca variância no eixo Z

# Criar cores para visualização (baseado na posição ao longo da tendência)
colors_proj = t

# Criar visualização
fig = plt.figure(figsize=(18, 10))

# ===== LINHA SUPERIOR: DADOS 3D COM PLANOS DE PROJEÇÃO =====

# Plot 1: Dados 3D com plano XY destacado (MAIOR VARIÂNCIA)
ax1 = fig.add_subplot(2, 2, 1, projection='3d')
ax1.scatter(X_proj[:, 0], X_proj[:, 1], X_proj[:, 2], 
           c=colors_proj, cmap='viridis', s=25, alpha=0.7, edgecolors='k', linewidth=0.3)
# Desenhar plano XY (z=0)
x_range = [X_proj[:, 0].min()-2, X_proj[:, 0].max()+2]
y_range = [X_proj[:, 1].min()-1, X_proj[:, 1].max()+1]
xx, yy = np.meshgrid(np.linspace(*x_range, 10), np.linspace(*y_range, 10))
zz = np.zeros_like(xx)
ax1.plot_surface(xx, yy, zz, alpha=0.25, color='red', edgecolor='none')
ax1.set_xlabel('X (forte tendência)', fontsize=11, labelpad=8)
ax1.set_ylabel('Y (var. média)', fontsize=11, labelpad=8)
ax1.set_zlabel('Z (var. mínima)', fontsize=11, labelpad=8)
ax1.set_title('Plano XY - Captura Tendência Principal', fontsize=13, pad=10)
ax1.view_init(elev=18, azim=35)

# Plot 2: Dados 3D com plano YZ destacado (MENOR VARIÂNCIA)
ax2 = fig.add_subplot(2, 2, 2, projection='3d')
ax2.scatter(X_proj[:, 0], X_proj[:, 1], X_proj[:, 2], 
           c=colors_proj, cmap='viridis', s=25, alpha=0.7, edgecolors='k', linewidth=0.3)
# Desenhar plano YZ (x=0)
y_range = [X_proj[:, 1].min()-1, X_proj[:, 1].max()+1]
z_range = [X_proj[:, 2].min()-0.5, X_proj[:, 2].max()+0.5]
yy, zz = np.meshgrid(np.linspace(*y_range, 10), np.linspace(*z_range, 10))
xx = np.zeros_like(yy)
ax2.plot_surface(xx, yy, zz, alpha=0.25, color='orange', edgecolor='none')
ax2.set_xlabel('X (forte tendência)', fontsize=11, labelpad=8)
ax2.set_ylabel('Y (var. média)', fontsize=11, labelpad=8)
ax2.set_zlabel('Z (var. mínima)', fontsize=11, labelpad=8)
ax2.set_title('Plano YZ - Perde Tendência Principal', fontsize=13, pad=10)
ax2.view_init(elev=18, azim=35)

# ===== LINHA INFERIOR: PROJEÇÕES 2D =====

# Calcular variâncias
var_x, var_y, var_z = np.var(X_proj[:, 0]), np.var(X_proj[:, 1]), np.var(X_proj[:, 2])
var_total = var_x + var_y + var_z
var_xy = (var_x + var_y) / var_total
var_yz = (var_y + var_z) / var_total

# Plot 3: Projeção no plano XY (PRESERVA MÁXIMA VARIÂNCIA)
ax3 = fig.add_subplot(2, 2, 3)
scatter3 = ax3.scatter(X_proj[:, 0], X_proj[:, 1], c=colors_proj, 
                      cmap='viridis', s=25, alpha=0.7, edgecolors='k', linewidth=0.3)
ax3.set_xlabel('X (forte tendência)', fontsize=11)
ax3.set_ylabel('Y (variância média)', fontsize=11)
ax3.set_title(f'Projeção XY - {var_xy*100:.2f}% da variância preservada', fontsize=13)
ax3.grid(True, alpha=0.3)
ax3.set_aspect('equal', adjustable='box')

# Plot 4: Projeção no plano YZ (PERDE MÁXIMA VARIÂNCIA)
ax4 = fig.add_subplot(2, 2, 4)
scatter4 = ax4.scatter(X_proj[:, 1], X_proj[:, 2], c=colors_proj, 
                      cmap='viridis', s=25, alpha=0.7, edgecolors='k', linewidth=0.3)
ax4.set_xlabel('Y (variância média)', fontsize=11)
ax4.set_ylabel('Z (variância mínima)', fontsize=11)
ax4.set_title(f'Projeção YZ - {var_yz*100:.2f}% da variância preservada', fontsize=13)
ax4.grid(True, alpha=0.3)
ax4.set_aspect('equal', adjustable='box')

plt.suptitle('Projeção 3D → 2D: Escolha do Plano Determina Informação Preservada', 
             fontsize=15, y=0.98, fontweight='bold')
plt.tight_layout()
plt.show()

### 3.2 Manifold Learning

Aqui a premissa é diferente: os dados de alta dimensão vivem em (ou próximos de) uma variedade de dimensão menor, mas essa variedade pode estar dobrada, torcida ou enrolada no espaço maior. A tarefa é "desenrolar" essa estrutura, preservando vizinhanças locais.

A analogia clássica é o rolo de papel higiênico: a superfície é 2D, mas está enrolada no espaço 3D. Métodos de manifold learning tentam "abrir" esse rolo e revelar a estrutura plana subjacente.

Exemplos incluem **LLE (Locally Linear Embedding)**, **t-SNE** e **Isomap**. Esses métodos capturam relações não-lineares e preservam estruturas locais, mas têm custo computacional maior, sensibilidade a hiperparâmetros e, em geral, não oferecem transformação explícita para novos dados.

A seguir, visualizamos ambos os cenários para consolidar a intuição.

In [ ]:
# ======= EXEMPLO: MANIFOLD LEARNING - SWISS ROLL =======

# Gerar Swiss Roll (papel higiênico enrolado)
X_swiss_demo, color_swiss_demo = make_swiss_roll(n_samples=1000, noise=0.05, random_state=42)

# Plotar o manifold 3D
fig = plt.figure(figsize=(18, 6))

# Visualização 3D do Swiss Roll
ax1 = fig.add_subplot(131, projection='3d')
scatter1 = ax1.scatter(X_swiss_demo[:, 0], X_swiss_demo[:, 1], X_swiss_demo[:, 2], 
                      c=color_swiss_demo, cmap='Spectral', s=20, alpha=0.7)
ax1.set_xlabel('X', fontsize=10)
ax1.set_ylabel('Y', fontsize=10)
ax1.set_zlabel('Z', fontsize=10)
ax1.set_title('Swiss Roll 3D - Superfície 2D Enrolada', fontsize=12)
ax1.view_init(elev=15, azim=70)
plt.colorbar(scatter1, ax=ax1, label='Posição ao longo do rolo', shrink=0.6)

# Visualização de outro ângulo
ax2 = fig.add_subplot(132, projection='3d')
scatter2 = ax2.scatter(X_swiss_demo[:, 0], X_swiss_demo[:, 1], X_swiss_demo[:, 2], 
                      c=color_swiss_demo, cmap='Spectral', s=20, alpha=0.7)
ax2.set_xlabel('X', fontsize=10)
ax2.set_ylabel('Y', fontsize=10)
ax2.set_zlabel('Z', fontsize=10)
ax2.set_title('Visão Lateral - Estrutura Enrolada Evidente', fontsize=12)
ax2.view_init(elev=0, azim=0)
plt.colorbar(scatter2, ax=ax2, label='Posição ao longo do rolo', shrink=0.6)

# Projeção simples (esmagamento) vs ideal (desenrolamento)
# Usar a coordenada original do swiss roll que representa o "desenrolamento"
ax3 = fig.add_subplot(133)
scatter3 = ax3.scatter(color_swiss_demo, X_swiss_demo[:, 1], 
                      c=color_swiss_demo, cmap='Spectral', s=20, alpha=0.7)
ax3.set_xlabel('Posição desenrolada (ideal)', fontsize=10)
ax3.set_ylabel('Altura (Y)', fontsize=10)
ax3.set_title('Estrutura 2D Revelada Após "Desenrolar"', fontsize=12)
ax3.grid(True, alpha=0.3)

plt.suptitle('Manifold Learning: Objetivo é "Desenrolar" Estruturas Não-Lineares', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

<a id='pca'></a>
## 4. PCA - Principal Component Analysis

<a id='pca-intuicao'></a>
### 4.1 Definição e Intuição

**PCA (Principal Component Analysis)** é a ferramenta mais popular e prática para redução de dimensionalidade linear. Sua ideia é simples: encontrar as direções no espaço original ao longo das quais os dados variam mais e descartar as direções onde há pouca variação.

Pense em um conjunto de pontos 3D formando uma elipse alongada. O PCA identifica o eixo maior da elipse como a **primeira direção principal** - a que captura a máxima variância. Depois, encontra uma segunda direção, perpendicular à primeira, que captura a máxima variância restante. E assim por diante.

Ao manter apenas os primeiros $k$ componentes principais, reduzimos a dimensionalidade de $n$ para $k$, preservando tipicamente 80-95% da informação original. Os algoritmos baseados em distância (como KNN ou K-Means) funcionam muito melhor neste espaço reduzido, onde apenas o sinal relevante domina.

O objetivo de PCA é transformar features correlacionadas (e muitas vezes redundantes) em features não-correlacionadas (ortogonais) que capturam máxima variância. Este é um achado fundamental: em um novo sistema de eixos bem escolhido, podemos descartar dimensões inteiras sem perder informação essencial.

<a id='pca-matematica'></a>
### 4.2 Formulação Matemática

O algoritmo de PCA segue uma sequência clara e bem-definida. Dado um dataset $\mathbf{X} \in \mathbb{R}^{m \times n}$ com $m$ amostras e $n$ features:

**Passo 1 - Centralizar os dados**  
Subtraímos a média de cada feature, garantindo que o espaço esteja centrado na origem:
$$\mathbf{X}_{\text{centered}} = \mathbf{X} - \boldsymbol{\mu}$$

**Passo 2 - Calcular a matriz de covariância**  
A matriz de covariância captura como cada par de features varia em conjunto:
$$\mathbf{C} = \frac{1}{m-1} \mathbf{X}_{\text{centered}}^T \mathbf{X}_{\text{centered}}$$

**Passo 3 - Decomposição espectral**  
Encontramos autovalores $\lambda_i$ e autovetores $\mathbf{v}_i$ de $\mathbf{C}$. Cada autovetor representa uma direção principal, e o correspondente autovalor representa a variância capturada por essa direção:
$$\mathbf{C} \mathbf{v}_i = \lambda_i \mathbf{v}_i$$

**Passo 4 - Selecionar componentes principais**  
Ordenamos os autovetores por autovalores em ordem decrescente. Os $k$ primeiros autovetores - aqueles com maiores autovalores - formam a matriz de projeção $\mathbf{W} \in \mathbb{R}^{n \times k}$.

**Passo 5 - Projetar dados**  
Finalmente, aplicamos a transformação ao dataset centralizado, gerando a representação reduzida:
$$\mathbf{X}_{\text{reduzido}} = \mathbf{X}_{\text{centered}} \mathbf{W}$$

**Métrica importante - Variância explicada**  
O percentual de variância capturado pelo componente $i$ é:
$$\frac{\lambda_i}{\sum_{j=1}^{n} \lambda_j}$$

Esta métrica ajuda a decidir quantos componentes são suficientes.

<a id='pca-implementacao'></a>
### 4.3 Implementação com Scikit-Learn

Vamos aplicar PCA no dataset **MNIST de dígitos** (8×8 pixels = 64 features) e reduzi-lo para 2D para visualização.

In [ ]:
# ======= CARREGAR DATASET DE DÍGITOS =======

digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"Shape original: {X_digits.shape}")
print(f"Número de classes: {len(np.unique(y_digits))}")
print(f"Pixels por imagem: {X_digits.shape[1]} (8x8)")

# Visualizar algumas imagens
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_digits[i].reshape(8, 8), cmap='gray')
    ax.set_title(f'Dígito: {y_digits[i]}')
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# ======= APLICAR PCA: 64D → 2D =======

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_digits)

print(f"Shape após PCA: {X_pca.shape}")
print(f"\nVariância explicada por componente:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.4f} ({var*100:.2f}%)")
print(f"\nVariância total explicada: {pca.explained_variance_ratio_.sum():.4f} ({pca.explained_variance_ratio_.sum()*100:.2f}%)")

In [ ]:
# ======= VISUALIZAÇÃO: PROJEÇÃO PCA 2D =======

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_digits, cmap='tab10', 
                     alpha=0.6, edgecolors='k', linewidth=0.5)
plt.colorbar(scatter, label='Dígito')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variância)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variância)', fontsize=12)
plt.title('PCA: Projeção de Dígitos de 64D para 2D', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#### Análise: Scree Plot e Variância Acumulada

Um aspecto crucial do PCA é decidir **quantos componentes manter**. Duas ferramentas comuns:

1. **Scree Plot**: Visualiza a variância explicada por cada componente
2. **Variância Acumulada**: Mostra quanta variância total é preservada ao manter os primeiros $k$ componentes

Geralmente buscamos manter 80-95% da variância original.

In [ ]:
# ======= ANÁLISE DE VARIÂNCIA: TODOS OS COMPONENTES =======

# Ajustar PCA com todos os componentes
pca_full = PCA()
pca_full.fit(X_digits)

# Calcular variância acumulada
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Criar subplots
fig, axes = plt.subplots(1, 2, figsize=(18, 5))

# Scree Plot
axes[0].bar(range(1, len(pca_full.explained_variance_ratio_) + 1), 
            pca_full.explained_variance_ratio_, alpha=0.7, color='steelblue')
axes[0].set_xlabel('Componente Principal', fontsize=12)
axes[0].set_ylabel('Variância Explicada', fontsize=12)
axes[0].set_title('Scree Plot: Variância por Componente', fontsize=14)
axes[0].grid(True, alpha=0.3, axis='y')

# Variância Acumulada
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
             marker='o', linewidth=2, markersize=4, color='darkgreen')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% variância', linewidth=2)
axes[1].axhline(y=0.90, color='orange', linestyle='--', label='90% variância', linewidth=2)
axes[1].set_xlabel('Número de Componentes', fontsize=12)
axes[1].set_ylabel('Variância Acumulada', fontsize=12)
axes[1].set_title('Variância Acumulada vs. Número de Componentes', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Encontrar número de componentes para 95% variância
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nComponentes necessários para 95% variância: {n_components_95}/{len(cumulative_variance)}")

#### Impacto da Redução no Desempenho de Modelos

Vamos treinar um classificador com diferentes números de componentes para ver como a redução afeta a acurácia.

In [ ]:
# ======= EXPERIMENTO: PCA + CLASSIFICADOR =======

# Split dos dados originais
X_train, X_test, y_train, y_test = train_test_split(
    X_digits, y_digits, test_size=0.2, random_state=42
)

# Testar diferentes números de componentes
n_components_list = [2, 5, 10, 15, 20, 30, 40, 50, 64]
accuracies = []

for n_comp in n_components_list:
    if n_comp <= X_train.shape[1]:
        # Aplicar PCA
        pca_temp = PCA(n_components=n_comp)
        X_train_pca = pca_temp.fit_transform(X_train)
        X_test_pca = pca_temp.transform(X_test)
        
        # Treinar classificador
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train_pca, y_train)
        
        # Avaliar
        acc = accuracy_score(y_test, clf.predict(X_test_pca))
        accuracies.append(acc)
        print(f"{n_comp:2d} componentes: {acc:.4f} (variância: {pca_temp.explained_variance_ratio_.sum():.3f})")

In [ ]:
# ======= VISUALIZAÇÃO: ACURÁCIA VS COMPONENTES =======

plt.figure(figsize=(12, 6))
plt.plot(n_components_list, accuracies, marker='o', linewidth=2, markersize=8, color='darkblue')
plt.xlabel('Número de Componentes PCA', fontsize=12)
plt.ylabel('Acurácia no Teste', fontsize=12)
plt.title('Desempenho do Classificador vs. Dimensionalidade Reduzida (PCA)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=max(accuracies), color='r', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### Visualização dos Componentes Principais

Os componentes principais são na verdade **direções no espaço original**. Podemos visualizá-los como "imagens" para entender o que cada componente captura.

In [ ]:
# ======= VISUALIZAR PRIMEIROS COMPONENTES COMO IMAGENS =======

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    if i < len(pca_full.components_):
        ax.imshow(pca_full.components_[i].reshape(8, 8), cmap='RdBu_r')
        ax.set_title(f'PC{i+1} ({pca_full.explained_variance_ratio_[i]*100:.1f}%)')
        ax.axis('off')
plt.suptitle('Primeiros 8 Componentes Principais (Autovetores)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show() 

<a id='lle'></a>
## 5. LLE - Locally Linear Embedding

<a id='lle-intuicao'></a>
### 5.1 Definição e Intuição

**LLE (Locally Linear Embedding)** representa uma mudança fundamental de perspectiva. Enquanto PCA busca direções globais de máxima variância, LLE assume que os dados de alta dimensão residem próximos a uma variedade (manifold) não-linear e desconhecida. Sua estratégia é preservar a **estrutura local** dos dados, mantendo vizinhanças intactas durante a redução.

A premissa é elegante: se dois pontos estão próximos no espaço original, eles devem permanecer próximos após a redução. Mais precisamente, cada ponto pode ser representado como uma combinação linear de seus vizinhos mais próximos. LLE tira vantagem disso: primeiro aprende os pesos que melhor reconstroem cada ponto a partir de seus vizinhos no espaço original; depois encontra uma projeção de baixa dimensão que preserva esses mesmos pesos.

O resultado é uma abordagem capaz de "desenrolar" manifolds complexos, capturando estruturas não-lineares que PCA jamais conseguiria explorar.

**O Algoritmo em Três Fases**:

**Fase 1 - Encontrar vizinhanças**  
Para cada ponto $\mathbf{x}_i$, identificamos seus $k$ vizinhos mais próximos baseado em distância euclidiana.

**Fase 2 - Aprender pesos locais**  
Encontramos os pesos $w_{ij}$ que melhor reconstroem cada ponto a partir de seus vizinhos, minimizando:
$$\min_{\mathbf{W}} \sum_i \left\| \mathbf{x}_i - \sum_j w_{ij} \mathbf{x}_j \right\|^2$$

**Fase 3 - Projetar preservando pesos**  
Encontramos projeções $\mathbf{y}_i$ em dimensão reduzida que mantêm esses mesmos pesos:
$$\min_{\mathbf{Y}} \sum_i \left\| \mathbf{y}_i - \sum_j w_{ij} \mathbf{y}_j \right\|^2$$

**Quando LLE brilha**: Em estruturas altamente não-lineares (como manifolds enrolados), LLE captura a geometria intrínseca melhor que qualquer método linear. Em estruturas lineares, ambos funcionam bem, mas PCA é mais rápido.

**Limitações práticas**: A escolha de $k$ (número de vizinhos) é crítica e sensível; não há transformação explícita, dificultando a aplicação a novos dados; é computacionalmente mais custoso que PCA.
- Mais lento que PCA

<a id='lle-implementacao'></a>
### 5.2 Implementação e Comparação com PCA

Vamos comparar PCA e LLE em dois cenários:
1. **Dataset de Dígitos**: Estrutura relativamente linear
2. **Swiss Roll**: Manifold não-linear clássico

#### Experimento 1: Dígitos (Estrutura Linear)

In [ ]:
# ======= APLICAR LLE NOS DÍGITOS =======

# Usar apenas subset para acelerar (LLE é mais lento)
n_samples_subset = 1000
indices = np.random.choice(len(X_digits), n_samples_subset, replace=False)
X_subset = X_digits[indices]
y_subset = y_digits[indices]

# PCA
pca_digits = PCA(n_components=2)
X_pca_digits = pca_digits.fit_transform(X_subset)

# LLE
lle_digits = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_lle_digits = lle_digits.fit_transform(X_subset)

In [ ]:
# ======= COMPARAÇÃO VISUAL: PCA VS LLE (DÍGITOS) =======

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# PCA
scatter1 = axes[0].scatter(X_pca_digits[:, 0], X_pca_digits[:, 1], c=y_subset, 
                          cmap='tab10', alpha=0.6, edgecolors='k', linewidth=0.5)
axes[0].set_xlabel('Componente 1', fontsize=12)
axes[0].set_ylabel('Componente 2', fontsize=12)
axes[0].set_title('PCA - Projeção Linear', fontsize=14)
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Dígito')

# LLE
scatter2 = axes[1].scatter(X_lle_digits[:, 0], X_lle_digits[:, 1], c=y_subset, 
                          cmap='tab10', alpha=0.6, edgecolors='k', linewidth=0.5)
axes[1].set_xlabel('Dimensão 1', fontsize=12)
axes[1].set_ylabel('Dimensão 2', fontsize=12)
axes[1].set_title('LLE - Manifold Learning (k=10 vizinhos)', fontsize=14)
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Dígito')

plt.suptitle('Comparação PCA vs LLE: Dataset de Dígitos', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

#### Experimento 2: Swiss Roll (Manifold Não-Linear)

O Swiss Roll é um dataset clássico em forma de "rolo suíço" - uma estrutura 2D enrolada no espaço 3D. É o caso ideal para demonstrar a superioridade de métodos de manifold learning.

In [ ]:
# ======= GERAR E VISUALIZAR SWISS ROLL =======

# Gerar Swiss Roll 3D
X_swiss, color_swiss = make_swiss_roll(n_samples=1500, noise=0.1, random_state=42)

# Visualizar em 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_swiss[:, 0], X_swiss[:, 1], X_swiss[:, 2], 
                    c=color_swiss, cmap='viridis', s=20, alpha=0.7)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Swiss Roll: Manifold 2D no Espaço 3D', fontsize=14)
plt.colorbar(scatter, label='Posição ao longo do rolo')
plt.tight_layout()
plt.show()

In [ ]:
# ======= APLICAR PCA E LLE NO SWISS ROLL =======

# PCA
pca_swiss = PCA(n_components=2)
X_pca_swiss = pca_swiss.fit_transform(X_swiss)

# LLE
lle_swiss = LocallyLinearEmbedding(n_components=2, n_neighbors=12, random_state=42)
X_lle_swiss = lle_swiss.fit_transform(X_swiss)

In [ ]:
# ======= COMPARAÇÃO VISUAL: PCA VS LLE (SWISS ROLL) =======

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# PCA
scatter1 = axes[0].scatter(X_pca_swiss[:, 0], X_pca_swiss[:, 1], c=color_swiss, 
                          cmap='viridis', alpha=0.7, s=20)
axes[0].set_xlabel('PC1', fontsize=12)
axes[0].set_ylabel('PC2', fontsize=12)
axes[0].set_title('PCA - Projeção Linear (Falha em "Desenrolar")', fontsize=14)
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Posição')

# LLE
scatter2 = axes[1].scatter(X_lle_swiss[:, 0], X_lle_swiss[:, 1], c=color_swiss, 
                          cmap='viridis', alpha=0.7, s=20)
axes[1].set_xlabel('Dimensão 1', fontsize=12)
axes[1].set_ylabel('Dimensão 2', fontsize=12)
axes[1].set_title('LLE - Desenrola o Manifold com Sucesso', fontsize=14)
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Posição')

plt.suptitle('Comparação PCA vs LLE: Swiss Roll (Manifold Não-Linear)', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### Interpretação das Comparações

**Dataset de Dígitos**: Ambos PCA e LLE funcionam razoavelmente bem, pois a estrutura é relativamente linear. PCA é preferível por ser mais rápido e ter transformação explícita.

**Swiss Roll**: 
- **PCA falha** em "desenrolar" o manifold - vemos apenas uma projeção achatada onde pontos distantes ao longo do rolo aparecem próximos
- **LLE consegue** preservar a estrutura local e desenrolar o rolo, revelando a estrutura 2D verdadeira

**Quando usar cada método**:
- **PCA**: Dados com variações principalmente lineares, necessidade de velocidade, aplicação a novos dados
- **LLE**: Dados com estrutura não-linear conhecida, foco em preservar relações locais, visualização exploratória

<a id='outros-metodos'></a>
## 6. Outros Métodos de Redução de Dimensionalidade

Além de PCA e LLE, o ecossistema oferece diversos outros métodos especializados, cada um adequado para cenários e dados específicos. Aqui apresentamos um panorama dos mais relevantes.

### 6.1 t-SNE (t-Distributed Stochastic Neighbor Embedding)

**t-SNE** é especialista em **visualização exploratória**. Ao contrário de PCA e LLE que buscam preservar variância ou vizinhança em geral, t-SNE otimiza especificamente para renderizar estruturas de cluster em 2D ou 3D de forma legível ao olho humano.

O algoritmo preserva as relações locais de proximidade com fidelidade notável: pontos próximos no espaço original tendemos a permanecer próximos na visualização, e clusters bem separados aparecem isolados visualmente. Isso o torna excelente para análise exploratória inicial de datasets.

**Limitações importantes**: t-SNE não é determinístico (resultados variam entre execuções), é computacionalmente muito lento para grandes datasets e não oferece transformação explícita, impossibilitando sua aplicação direto a novos dados. Também é sensível a hiperparâmetros como taxa de aprendizado e "perplexidade".

**Extra**: [`sklearn.manifold.TSNE`](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)

### 6.2 Isomap

**Isomap** estende o conceito clássico de MDS (Multidimensional Scaling) para dados em manifolds não-lineares. Sua inovação principal é usar **distâncias geodésicas** (caminho mais curto *ao longo* da variedade) em vez de distâncias euclidianas diretas. Isto permite capturar a geometria intrínseca de manifolds complexos.

O algoritmo: (1) constrói um grafo de vizinhança dos pontos; (2) calcula distâncias geodésicas via shortest paths no grafo; (3) aplica MDS clássico nessas distâncias. É particularmente eficaz em manifolds bem-definidos sem "furos" ou topologias complexas.

**Limitação**: Sensível a ruído e descontinuidades nos dados; a qualidade depende criticamente de encontrar a vizinhança correta.

**Extra**: [`sklearn.manifold.Isomap`](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.Isomap.html)

### 6.4 Kernel PCA

**Kernel PCA** transporta o "kernel trick" - familiar de SVM - para o contexto de PCA. Ao invés de trabalhar no espaço original de features, Kernel PCA implicitamente aplica uma transformação não-linear (via kernel) e então executa PCA nesse novo espaço transformado.

Isso permite capturar estruturas não-lineares que PCA linear seria incapaz de revelar. A escolha do kernel (RBF, polinomial, sigmoid) e seus hiperparâmetros é crítica e requer validação cuidadosa. Em muitos cenários, oferece melhor compromisso entre simplicidade de PCA e flexibilidade de métodos manifold.
C
**Extra**: [`sklearn.decomposition.KernelPCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html)

### 6.5 Autoencoders (Deep Learning)

**Autoencoders** são redes neurais que aprendem representações comprimidas de forma completamente não-supervisionada. A arquitetura é simétrica: um encoder que comprime dados em dimensão reduzida, seguido de um decoder que reconstrói os dados originais. A perda de reconstrução força a aprendizagem de representações informativas.

O poder dos autoencoders está em sua flexibilidade: variantes como variational autoencoders (VAE) ou convolutional autoencoders (para imagens) permitem capturar estruturas altamente não-lineares e complexas. São especialmente valiosos em domínios como imagens, áudio e texto.

**Limitações**: Requerem muito mais dados e poder computacional que métodos clássicos; ajuste de hiperparâmetros (arquitetura, taxa de aprendizado) é mais envolvido; interpretação das dimensões latentes é menos clara.

---

<-- [**Anterior: Introdução ao Unsupervised Learning**](01_introducao.ipynb) | [**Próximo: Clustering**](03_clustering.ipynb) -->